<a href="https://colab.research.google.com/github/bloodpet/eman-misc/blob/master/gpt_reviews_l1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Sep 11 13:43:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Init

In [1]:
### DO Spaces keys
import getpass
access_key = getpass.getpass("access key: ")
secret_key = getpass.getpass("secret key: ")

access key: ··········
secret key: ··········


In [1]:
### import, install, define
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import requests
import getpass
import boto3
from dataclasses import dataclass
import tarfile
import glob
from datetime import datetime


run_name = "reviews5"
run_name = "reviews_l1"
dataset = f"{run_name}.txt"

#model_name = "124M"
#model_name = "355M"
model_name = "774M"
#model_name = "1558M"

steps = 25000
steps = 10000  # 5000  # default: 1000
sample_every = steps  # default: 200
save_every = 1000  # default: 500


@dataclass
class Settings:
  AWS_ACCESS_KEY_ID: str
  AWS_SECRET_ACCESS_KEY: str
  AWS_S3_PREFIX: str = run_name
  AWS_STORAGE_BUCKET_NAME: str = "eman-space"
  AWS_S3_ENDPOINT_URL: str = "https://sgp1.digitaloceanspaces.com"
  AWS_REGION_NAME: str = "sgp1"


settings = Settings(
    access_key,
    secret_key,
)


class S3(object):
    bucket = settings.AWS_STORAGE_BUCKET_NAME
    endpoint = settings.AWS_S3_ENDPOINT_URL

    def __init__(self):
        self.session = boto3.session.Session()

    @property
    def client(self):
        return self.session.client(
            "s3",
            region_name=settings.AWS_REGION_NAME,
            endpoint_url=self.endpoint,
            aws_access_key_id=settings.AWS_ACCESS_KEY_ID,
            aws_secret_access_key=settings.AWS_SECRET_ACCESS_KEY,
        )

    def download(self, filename):
        remote_name = self.get_key(filename)
        self.client.download_file(
            self.bucket, remote_name, filename,
        )
        return self.client.head_object(Bucket=self.bucket, Key=remote_name)

    def get_key(self, filename):
        if settings.AWS_S3_PREFIX:
            return f"{settings.AWS_S3_PREFIX}/{filename}"
        else:
            return filename

    def upload(self, filename, extra_args=None):
        if extra_args is None:
            extra_args = {}
        extra_args["ACL"] = "public-read"
        remote_name = self.get_key(filename)
        self.client.upload_file(
            filename, self.bucket, remote_name, ExtraArgs=extra_args,
        )
        return remote_name

    def upload_content(self, filename, content):
        remote_name = self.get_key(filename)
        self.client.upload_fileobj(
            content, self.bucket, remote_name, ExtraArgs={"ACL": "public-read"},
        )
        return remote_name

    def delete(self, key):
        self.client.delete_object(Bucket=self.bucket, Key=key)


s3 = S3()


def download_file_from_s3(filename):
  s3.download(filename)


def download_http(url, filename):
  resp = requests.get(url)
  with open(filename, "r") as fp:
    fp.write(resp.text)


def upload_checkpoint_to_s3():
  tar_file = f"{run_name}.tar"
  with tarfile.open(tar_file, "w") as fp:
    for filename in glob.glob(f"checkpoint/{run_name}/*"):
      fp.add(filename)
  s3.upload(tar_file)

def download_checkpoint_from_s3():
  tar_file = f"{run_name}.tar"
  s3.download(tar_file)
  with tarfile.open(tar_file, "r") as fp:
    fp.extractall()

def show_time():
  print("time:", datetime.now())

def finetune(session, download=True, restore_from='latest'):
  if download:
    print("Download")
    show_time()
    download_checkpoint_from_s3()
  print("Finetune")
  show_time()
  gpt2.finetune(
      sess,
      dataset=dataset,
      model_name=model_name,
      steps=steps,
      restore_from=restore_from,
      run_name=run_name,
      print_every=10,
      sample_every=sample_every,
      save_every=save_every,
  )
  show_time()
  print("Upload")
  upload_checkpoint_to_s3()
  show_time()
  print("Done")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



NameError: ignored

In [3]:
### Download & start session
show_time()
#gpt2.download_gpt2(model_name=model_name)
#s3.download(dataset)
print("Start session")
sess = gpt2.start_tf_sess()
show_time()

time: 2020-09-11 14:01:35.370989
Start session
time: 2020-09-11 14:01:35.602141


# Scratch

In [ ]:
print("Download")
show_time()
download_checkpoint_from_s3()

Download
time: 2020-09-11 13:04:30.071090


In [ ]:
show_time()

time: 2020-09-11 13:06:07.100112


In [ ]:

### Notes
* help: ctrl+m h
* change to code: ctrl+m y
* change to text: ctrl+m m

#### reviews5 steps 0-5000 1st try
* start: 4am
* end: 7:30am

#### reviews5 steps 0-5000 2nd try
* start: 2020-09-04 23:50:08.786472 (7:50)
* loss=2.59 avg=2.59
* end: 2020-09-05 02:13:41.456083
* loss=2.01 avg=2.56

#### reviews5 steps 5000-10000
* dl: 2020-09-05 04:36:31.018127 (12:35)
* train: 2020-09-05 04:37:33.159364
* loss=2.42 avg=2.42
* end: 2020-09-05 06:57:40.393788
* loss=2.44 avg=2.43
* ul end: 2020-09-05 06:58:54.167808

#### reviews5 steps 10000-15000
* dl: 2020-09-05 13:59:16.498964
* train: 2020-09-05 14:00:15.432836
* init: loss=2.26 avg=2.26
* end: 2020-09-05 16:17:09.752531
* result: loss=3.03 avg=2.31
* ul end: 2020-09-05 16:18:23.648765

#### reviews5 steps 15000-20000
* dl: 
* train: 2020-09-05 16:56:32.091507
* init: loss=2.75 avg=2.75
* end: 2020-09-05 19:13:44.524913
* result: loss=2.33 avg=2.14
* ul end: 2020-09-05 19:13:44.525202

#### reviews5 steps 20000-30000
* dl: 
* train: 2020-09-05 19:23:26.423300
* init: loss=2.69 avg=2.69
* end: 2020-09-05 23:54:05.237514
* result: loss=1.81 avg=1.86
* ul end: 2020-09-05 23:54:05.237789

#### reviews5 steps 30000-40000
* dl: 2020-09-06 03:23:38.180403
* train: 2020-09-06 03:24:31.035087
* init: loss=0.91 avg=0.91
* end: 2020-09-06 07:58:17.759233
* result: loss=1.44 avg=1.66
* ul end: 2020-09-06 07:59:43.613905

#### reviews5 steps 40000-50000
* dl: 2020-09-06 11:24:46.673572
* train: 2020-09-06 11:25:26.815786
* init: loss=2.00 avg=2.00
* end: 2020-09-06 16:09:17.421952
* result: loss=2.16 avg=1.49
* ul end: 2020-09-06 16:10:15.233178

#### reviews5 steps 5k-75k
* dl: 2020-09-07 03:39:38.657501
* train: 2020-09-07 03:40:41.717020
* init: loss=0.34 avg=0.34 / loss=1.57 avg=0.96
* result: loss=1.64 avg=0.99
* end: 2020-09-07 09:57:43.213194
* ul end: 2020-09-07 10:06:22.822548

#### reviews5 steps 75k-100k (failed)
* dl: 
* train: 2020-09-07 11:31:12.469546
* init: loss=1.82 avg=1.82 / loss=1.06 avg=1.44
* result: 
* end: 
* ul end: 

#### reviews5 steps 75k-100k
* dl: 2020-09-08 02:01:31.879552
* train: 2020-09-08 02:02:13.639671
* init: loss=0.52 avg=0.52 / loss=1.03 avg=0.77
* result: loss=0.65 avg=0.68 / loss=0.75 avg=0.68
* end: 2020-09-08 08:16:22.630328
* ul end: 2020-09-08 08:18:49.124201

#### reviews5 steps 100k-125k (failed)
* dl: 2020-09-09 09:57:59.637810
* train: 2020-09-09 09:58:42.786696
* init: loss=0.36 avg=0.36 / loss=1.15 avg=0.76 / loss=0.29 avg=0.60
* result: 
* end: 
* ul end: 

#### reviews5 steps 125k-150k (failed)
* dl: 2020-09-10 16:21:28.306271
* train: 
* init: 
* result: 
* end: 
* ul end: 

#### reviews5 steps 125k-150k
* dl: 2020-09-11 00:56:01.183413
* train: 2020-09-11 00:56:31.983836
* init: loss=0.70 avg=0.70 / loss=0.38 avg=0.54
* result: 
* end: 
* ul end: 

#### reviews5 steps 125k-150k
* dl: 
* train: 
* init: 
* result: 
* end: 
* ul end: 


# Persist

In [5]:
##steps = 25000
## keep notebook running
aoeusnth = steps
## hope this works
aoeu = steps
## please work
snth = steps
## comments are to make sure of multiple lines in text mode
aoeu = snth
snth = steps
snth = aoeu
aoeu = steps
print(steps)

10000


In [ ]:
steps = 25000
#gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 391Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 74.3Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 356Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:10, 138Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 233Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 97.7Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 137Mit/s]                                                       


In [4]:
### Start from existing downloaded session
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every,
              )
show_time()
print("Upload")
show_time()
upload_checkpoint_to_s3()
print("Done")

Finetune
time: 2020-09-11 14:01:44.290540
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.


ResourceExhaustedError: ignored

# Logs reviews_l1


#### steps 125k-150k
* dl: 
* train: 2020-09-11 14:01:44.290540
* init: 
* result: 
* end: 
* ul end: 

#steps = 25000
# keep notebook running
aoeusnth = steps
# hope this works
aoeu = steps
# please work
snth = steps
# comments are to make sure of multiple lines in text mode
aoeu = snth
snth = steps
snth = aoeu
aoeu = steps
print(steps)

In [ ]:
### Start from existing session 100k - 125k failed
print("Download")
show_time()
download_checkpoint_from_s3()
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
upload_checkpoint_to_s3()
show_time()
print("Done")

Download
time: 2020-09-09 09:57:59.637810
Finetune
time: 2020-09-09 09:58:42.786696
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-100000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-100000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:59<00:00, 59.17s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-100000
======== SAMPLE 1 ========
what you can do with the phone is talk, do drawings, do digital drawing or digital painting or digital art, and so on. To do all that, they need the phone.”
If it sounds like a bunch of talk, no one has ever stopped laughing in the room; only a few people will have the confidence and balls to do so. As much as it’s easy to sellie and pulled-up to the whole world with the Nexus 7P, there just isn’t anything quite like this no-brainer. But the new Nexus 7 works just fine, and will make a powerful mid-range phone entirely accessible for some people.
The Nexus 7P launch price is $399 / £399 / AU$769, which is pretty steep for a smartphone with no Bixby smarts bolted in by default (the Google Home app also offers a free 14-day trial). The Nexus 6P had a rear fingerprint sensor for double the cost of the Nexus 7P, but we’d rather have the Nexus 7P as it has an inferior display, worse ch

In [ ]:
### Start from existing session 75k - 100k
print("Download")
show_time()
download_checkpoint_from_s3()
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
upload_checkpoint_to_s3()
show_time()
print("Done")

Download
time: 2020-09-08 02:01:31.879552
Finetune
time: 2020-09-08 02:02:13.639671
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-75000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-75000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:56<00:00, 56.49s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-75000
======== SAMPLE 1 ========
-like experience with a full keyboard and of course Windows 10. For us, however, anything other than a wired keyboard and mouse is a bit more awkward and therefore less attractive, a little outdated, than the recent trend to push for ultra-portable.
While you’ll be delighted by the BlackBerry KeyOne keyboard, it does make for an unsatisfying range of typing experience. We certainly had confidence in our fingers typing fast and consistently, using large fonts and numbers, just as we would on a laptop or phone. However, being all-plastic like this is no use for the majority of consumers who have been steadily abandoning traditional keyboard input as we drift ever smaller in desktop real estate.
Back to that Windows 10 keyboard, for once.
It looks very similar to the current Lenovo ThinkPad X1 Yoga above. However, on this K10 keyboard you’ll find a set of mechanical switches that have

In [ ]:
### Start from existing session 50k - 75k
print("Download")
show_time()
download_checkpoint_from_s3()
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
upload_checkpoint_to_s3()
show_time()
print("Done")

Download
time: 2020-09-07 03:39:38.657501
Finetune
time: 2020-09-07 03:40:41.717020
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-50000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-50000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [01:08<00:00, 68.28s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-50000
======== SAMPLE 1 ========
 pre-package or in the original box.
The instructions are clear and easy to understand, and if you ever mess up the procedure, there are clear and unambiguous examples of what to do if you go wrong (which happened more or less every time with every bag we tested).
Our testers also uniformly found the materials of the Tumi Bags to be thicker and warmer looking than those of the other nylon bags we tested.
The Tumi Original is made of canvas, so it's a good option for heavy gear, like this pair of binoculars stuffed with hiking gear.
Pockets and organization: The Tumi Original has some of the best storage among the bags we tried, with two large, Velcro-tight exterior pockets, plus a third inner zippered slot for sunglasses and a medium pouch for phone or notebook.
You also get two additional sets of shoulder straps in addition to the two you get in the regular bag.
Tumi also places a

In [ ]:
### Start from existing downloaded session 50k - 60k
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
show_time()
upload_checkpoint_to_s3()
print("Done")

Finetune
time: 2020-09-06 16:37:40.572292
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-50000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-50000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:54<00:00, 54.51s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-50000
======== SAMPLE 1 ========
 different, in theory, to allow a wider range of pressure settings.
The Dyson has a wide range of pressure settings.
Unlike some of its competitors, the Dyson has a wide and expansive range of pressure settings, from the softest setting of 15 to the intense highest setting you can find on the high-contrast Platinum Elite.
The Dyson’s dual sensors also let us know, with a very fiddly system: Press the button, and you get an alert, but you can't turn it off—we accidentally bumped the button and released the lock.
If you don’t release the lock immediately (as you might when you’re changing from an animal to an organic feed), you may break your finger in the same place before the alert subsides.
It’s the same basic mechanism as the other models, but it’s an improvement.
If you find yourself frequently frustrated by having to tap a button to get the Dyson to start cleaning, there’s also

In [ ]:
### Start from existing session 40k-50k
print("Download")
show_time()
download_checkpoint_from_s3()
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
upload_checkpoint_to_s3()
show_time()
print("Done")

Download
time: 2020-09-06 11:24:46.673572
Finetune
time: 2020-09-06 11:25:26.815786
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-40000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-40000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:55<00:00, 55.50s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-40000
======== SAMPLE 1 ========
 factor than with smaller-size models.
That’s not, by any means, a bad thing.
Smaller hands have much more dexterity than larger ones, and larger ones have much more dexterity than small-sized hands.
If your hands are too big for the Hahn 4.25-inch pot, try our other picks at our other recommendations below.
A few things set apart the 4.25-inch model from the typical 10-inch kettle.
For one thing, it’s the heaviest pot of the three tested at 2.7 pounds (2.86 KG).
That can easily scare kids with its heft.
When we tested it, we wondered if it might overheat.
After all those stirs and fillings, it didn’t have any brown spots.
With its heft and heavy-handed-ness, the 4.25-inch model can intimidate small hands and adults; for me, it was quite literally the biggest pot we tested, but because I could hold it without spilling a drop, it was a breeze to push with one hand.
The other thing t

In [ ]:
### Start from existing session 30k-40k
print("Download")
show_time()
download_checkpoint_from_s3()
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
upload_checkpoint_to_s3()
show_time()
print("Done")

Download
time: 2020-09-06 03:23:38.180403
Finetune
time: 2020-09-06 03:24:31.035087
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-30000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-30000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:57<00:00, 57.35s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-30000
======== SAMPLE 1 ========
 in a backcountry location.
While most hiking lights in the test group flashed to a full flash, we found that the Arc’teryx Surge was the most difficult to see when light sources were illuminating behind it.
You have to really look for it, either to find it, or to find it plus other lights in a backcountry location.
This was a test of the headlamp brightness and intensity, and we didn’t have a good way to measure the effect that the surges meant to have on the quality of our own sleep.
As an example: As a pack-in light, both the Surge LED and the OBD-II lights were brighter than the CR2 for the last hour, and in the last mile in the high desert of New Mexico we saw them blinking.
In fact, during that last mile, our top light pick, the Sony SmartBand FreeLight, flashed twice as bright as the Surge and was four times brighter than the CR2.
So there was something about the Surge that 

In [ ]:
### Start from existing downloaded session 20k - 30k
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
show_time()
upload_checkpoint_to_s3()
print("Done")

Finetune
time: 2020-09-05 19:23:26.423300
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-20000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-20000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:55<00:00, 55.25s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-20000
======== SAMPLE 1 ========
 dips.
It’s important to choose a pillow that you think will work for all sleep positions.
The American Academy of Pediatrics, in its parents’ pillows for kids and adults study, advises against sleep on a mattress of your own because it can shift positions, so you may fall asleep with your head leaning forward or backward.
To avoid this risk, always make sure your torso is in the correct position between the mattress and your knees.
You may also want to keep your mattress on its side—on the firm side—so that the mattress doesn’t feel too warm.
That way, the heat from the air mattress can cause you to be more susceptible to overheating than if you preferred the firm side of a mattress.
A good rule of thumb for getting the most bang for your buck is: Always try to choose the softest mattress so you don’t have or miss the sensation of sinking in.
If there is such a thing as “too soft”

In [ ]:
### Start from existing downloaded session 20k-30k
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='latest',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
show_time()
upload_checkpoint_to_s3()
print("Done")

Finetune
time: 2020-09-05 19:23:26.423300
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint checkpoint/reviews5/model-20000
INFO:tensorflow:Restoring parameters from checkpoint/reviews5/model-20000


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:55<00:00, 55.25s/it]


dataset has 12543225 tokens
Training...
Saving checkpoint/reviews5/model-20000
======== SAMPLE 1 ========
 dips.
It’s important to choose a pillow that you think will work for all sleep positions.
The American Academy of Pediatrics, in its parents’ pillows for kids and adults study, advises against sleep on a mattress of your own because it can shift positions, so you may fall asleep with your head leaning forward or backward.
To avoid this risk, always make sure your torso is in the correct position between the mattress and your knees.
You may also want to keep your mattress on its side—on the firm side—so that the mattress doesn’t feel too warm.
That way, the heat from the air mattress can cause you to be more susceptible to overheating than if you preferred the firm side of a mattress.
A good rule of thumb for getting the most bang for your buck is: Always try to choose the softest mattress so you don’t have or miss the sensation of sinking in.
If there is such a thing as “too soft”

In [ ]:
### Start fresh
print("Finetune")
show_time()
gpt2.finetune(sess,
              dataset=dataset,
              model_name=model_name,
              steps=steps,
              restore_from='fresh',
              run_name=run_name,
              print_every=10,
              sample_every=sample_every,
              save_every=save_every
              )
show_time()
print("Upload")
upload_checkpoint_to_s3()
show_time()
print("Done")

Finetune
time: 2020-09-04 23:50:08.786472
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:55<00:00, 55.27s/it]


dataset has 12543225 tokens
Training...
[10 | 23.71] loss=2.59 avg=2.59
[20 | 39.98] loss=2.53 avg=2.56
[30 | 56.54] loss=3.06 avg=2.73
[40 | 73.79] loss=3.08 avg=2.82
[50 | 90.95] loss=2.04 avg=2.66
[60 | 107.67] loss=3.13 avg=2.74
[70 | 124.19] loss=2.59 avg=2.72
[80 | 140.85] loss=2.68 avg=2.71
[90 | 157.75] loss=2.88 avg=2.73
[100 | 174.68] loss=2.88 avg=2.75
[110 | 191.45] loss=2.92 avg=2.76
[120 | 208.23] loss=2.91 avg=2.78
[130 | 224.96] loss=2.76 avg=2.78
[140 | 241.77] loss=3.04 avg=2.80
[150 | 258.64] loss=2.98 avg=2.81
[160 | 275.52] loss=2.89 avg=2.81
[170 | 292.39] loss=2.40 avg=2.79
[180 | 309.21] loss=3.08 avg=2.81
[190 | 326.04] loss=2.82 avg=2.81
[200 | 342.80] loss=2.66 avg=2.80
[210 | 359.61] loss=2.83 avg=2.80
[220 | 376.45] loss=2.91 avg=2.81
[230 | 393.22] loss=2.79 avg=2.81
[240 | 410.01] loss=3.03 avg=2.82
[250 | 426.81] loss=2.95 avg=2.82
[260 | 443.60] loss=3.02 avg=2.83
[270 | 460.39] loss=2.97 avg=2.84
[280 | 477.17] loss=2.28 avg=2.81
[290 | 493.99] loss=2.